In [3]:
import tensorflow as tf
from tensorflow import keras

DATA_PATH = 'data/Camera 2/test'
NUM_CLASSES = 10
BATCH_SIZE = 64

In [4]:
import glob

images_path, class_ids = [], []
for image_path in glob.glob(DATA_PATH + '/*/*'):
    class_id = image_path.split('/')[-2][-1]
    images_path.append(image_path)
    class_ids.append(class_id)
    
len(class_ids)

800

In [5]:
def preprocess(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = keras.applications.mobilenet.preprocess_input(img)
    print(img)
    return img, int(label)

In [6]:
data = tf.data.Dataset.from_tensor_slices((images_path, class_ids))

2021-12-12 21:47:17.747245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 21:47:17.846666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 21:47:17.847924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 21:47:17.862585: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
data = data.map(preprocess, num_parallel_calls=6).batch(BATCH_SIZE).prefetch(1)

Tensor("sub:0", shape=(224, 224, 3), dtype=float32)


In [8]:
model = keras.Sequential([
    keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(NUM_CLASSES)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                10250     
                                                                 
Total params: 3,239,114
Trainable params: 3,217,226
Non-trainable params: 21,888
_________________________________________________________________


In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.evaluate(data)

2021-12-12 21:48:39.640219: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2021-12-12 21:48:41.014414: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


13/13 [==============================] - 15s 648ms/step - loss: 10.0240 - accuracy: 0.1025


[10.023987770080566, 0.10249999910593033]